In [ ]:
import numpy as np
import pandas as pd 

In [ ]:
df = pd.read_csv("../input/kwproject/kwprojb_data.csv")
df

In [ ]:
df['shop_type'].unique()

In [ ]:
df['shop_type'].nunique()

In [ ]:
A = df[df['shop_type'] == '세탁소'].index

In [ ]:
df.drop(A,axis='index',inplace=True)
df

In [ ]:
df['shop_type'].nunique()

In [ ]:
df['shop_type'].unique()

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['shop_type'] = le.fit_transform(list(df['shop_type']))

In [ ]:
le.classes_

In [ ]:
df

In [ ]:
# df['shop_type'] = df['shop_type'].replace({'고기 뷔페':0,'고기구이/육회 전문점':1,'곱창/양구이 전문점':2,'구내식당':3,'국수/수제비 전문점':4,'기타세계요리 전문점':5,'낙지/문어/오징어/쭈꾸미 전문점':6,
#                                           '냉면 전문점':7,'덮밥/가정식':8,'도너츠/꽈배기/고로케/호떡 전문점':9,'도시락/컵밥/주먹밥/배달음식 전문점':10,'돈가스 전문점':11,'떡/한과 전문점':12,'마라/양꼬치/훠궈 전문점':13,
#                                           '면 전문점':14,'미분류 유흥업소':15,'민속주점':16,'바닷가재/게요리 전문점':17,'반찬 판매':18,'백화점':19,'버거 전문점':20,'베트남음식 전문점':21,'보쌈/족발 전문점':22,
#                                           '분식 전문점':23,'샌드위치/토스트/핫도그 전문점':24,'샤브샤브 전문점':25,'소주방/포장마차':26,'순대 전문점':27,'슈퍼/잡화/할인매장':28,'스테이크 전문점':29,
#                                           '식료품':30,'아이스크림/빙수 판매':31,'오리/닭요리 전문점':32,'오리/닭요리/탕/백숙 전문점':33,'와인/칵테일/위스키 바':34,'음식-미분류':35,'이자까야/꼬치구이':36,'이태리 전문점':37,
#                                           '인도음식 전문점':38,'일반 중식당':39,'장어 전문점':40,'전통찻집':41,'제과/제빵 전문점':42,'조개 전문점':43,'종합 뷔페':44,'죽요리 전문점':45,'참치 전문점':46,'초밥 전문점':47,
#                                           '치킨/닭강정 전문점':48,'카레 전문점':49,'카페/디저트':50,'테이크아웃 음료 전문점':51,'편의점':52,'퓨전음식 전문점':53,'피자 전문점':54,'한식국물요리 전문점':55,'한정식/백반 전문점':56,
#                                           '호프/맥주':57,'화로구이 전문점':58,'회/수산물 일반':59})

In [ ]:
X_column = ["shop_name"]
y_column = 'shop_type'

In [ ]:
input_data = df[X_column + [y_column]].copy()
input_data

In [ ]:
from sklearn.model_selection import train_test_split

tr_val_X, test_X, tr_val_y, test_y = train_test_split(input_data[X_column], input_data[y_column], test_size=0.2, shuffle=True, random_state=42, stratify=input_data[y_column])

In [ ]:
train_X, valid_X, train_y, valid_y = train_test_split(tr_val_X, tr_val_y, test_size=0.2, shuffle=True, random_state=42, stratify=tr_val_y)

In [ ]:
print(len(train_X))
print(len(valid_X))
print(len(test_X))
      
print(len(train_X) + len(valid_X) + len(test_X))
print(len(input_data))

In [ ]:
#train_X, valid_X, test_X를 각각 sequencing하고 padding 진행할 것!

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

tk = Tokenizer() 
tk.fit_on_texts(input_data['shop_name'])

In [ ]:
# 테스트용
seq_data = tk.texts_to_sequences(input_data['shop_name'])

from keras.preprocessing.sequence import pad_sequences
pad_seq_data = pad_sequences(seq_data)
pad_seq_data

In [ ]:
# nlp_inpul_length이 8인지 확인하려고 테스트용 진행
nlp_input_length = pad_seq_data[0].shape[0]
nlp_input_length

In [ ]:
max_len = 8 # nlp_input_length로 max 길이 확인

In [ ]:
# train, valid, test -> 정수인코딩 -> 패딩
seq_data = tk.texts_to_sequences(train_X['shop_name'])
pad_seq_data = pad_sequences(seq_data, max_len)
nlp_train_X = pad_seq_data

In [ ]:
seq_data = tk.texts_to_sequences(valid_X['shop_name'])
pad_seq_data = pad_sequences(seq_data, max_len)
nlp_valid_X = pad_seq_data

In [ ]:
seq_data = tk.texts_to_sequences(test_X['shop_name'])
pad_seq_data = pad_sequences(seq_data, max_len)
nlp_test_X = pad_seq_data

In [ ]:
nlp_test_X

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras

In [ ]:
class MultiHeadSelfAttention(layers.Layer):
    def __init__(self, embed_dim, num_heads=8):
        super(MultiHeadSelfAttention, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        if embed_dim % num_heads !=0:
             raise ValueError(
                f"embedding dimension = {embed_dim} should be divisible by number of heads = {num_heads}"
            )
        self.projection_dim = embed_dim // num_heads
        self.query_dense = layers.Dense(embed_dim)
        self.key_dense = layers.Dense(embed_dim)
        self.value_dense = layers.Dense(embed_dim)
        self.combine_heads = layers.Dense(embed_dim)
    
    def attention(self, query, key, value):
        score = tf.matmul(query, key, transpose_b = True)
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32)
        scaled_score = score / tf.math.sqrt(dim_key)
        weights = tf.nn.softmax(scaled_score, axis=-1)
        output = tf.matmul(weights, value)
        return output, weights
    
    def seperate_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm = [0, 2, 1, 3])
    
    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        query = self.query_dense(inputs)
        key = self.key_dense(inputs)
        value = self.value_dense(inputs)
        
        query = self.seperate_heads(query, batch_size)
        key = self.seperate_heads(key, batch_size)
        value = self.seperate_heads(value, batch_size)
        
        attention, weights = self.attention(query, key, value)
        
        attention = tf.transpose(attention, perm=[0,2,1,3])
        
        concat_attention = tf.reshape(attention, (batch_size, -1, self.embed_dim))
        
        output = self.combine_heads(concat_attention)
        
        return output

In [ ]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadSelfAttention(embed_dim, num_heads)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation='relu'), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)
    
    def call(self, inputs, training):
        attn_output = self.att(inputs)
        attn_output = self.dropout1(attn_output, training = training)
        
        out1 = self.layernorm1(inputs + attn_output)
        
        ffn_output = self.ffn(out1)
        
        fnn_output = self.dropout2(ffn_output, training = training)
        return self.layernorm2(out1 + ffn_output)

In [ ]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim = vocab_size, output_dim = embed_dim)
        self.pos_emb = layers.Embedding(input_dim = maxlen, output_dim = embed_dim)
    
    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit = maxlen, delta=1)
        positions = self.pos_emb(positions)
        
        x = self.token_emb(x)
        
        return x + positions

In [ ]:
# making all sentences of same length
maxlen = 8

# vocabulray size
vocab_size = len(tk.word_index) + 1

In [ ]:
n_classes = input_data['shop_type'].nunique()
n_classes

In [ ]:
num_heads = 4
ff_dim = 32
embed_dim = 100
    
inputs = layers.Input(shape = (maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)

x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(10, activation='relu')(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(n_classes, activation='softmax')(x)

model = keras.Model(inputs = inputs, outputs = outputs)

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
n_batches = 32
n_epochs = 20

hist = model.fit(nlp_train_X, train_y, validation_data=(nlp_valid_X, valid_y),
                 epochs = n_epochs, batch_size = n_batches)

In [ ]:
# categorical_crossentropy를 쓰면 0 1 2 3 숫자 그대로 인식
# -> number value가 아닌 categorical value로 인식해야 하기 때문에 sparse_categorical_crossentropy 사용!!!

In [ ]:
model.summary()

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize = (20,10))
ax = plt.axes()
ax.set_xticks(np.arange(0, n_epochs, 1))
ax.set_xticklabels(np.arange(1, n_epochs + 1, 1))
plt.plot(hist.history['loss'], label = 'train loss')
plt.plot(hist.history['val_loss'], label = 'validation loss')
plt.xticks(np.arange(0, n_epochs, 1))
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend(loc = 'lower right')
plt.show

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize = (20,10))
ax = plt.axes()
ax.set_xticks(np.arange(0, n_epochs, 1))
ax.set_xticklabels(np.arange(1, n_epochs + 1, 1))
plt.plot(hist.history['accuracy'], label = 'train accuracy')
plt.plot(hist.history['val_accuracy'], label = 'validation accuracy')
plt.xticks(np.arange(0, n_epochs, 1))
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend(loc = 'lower right')
plt.show

In [ ]:
hist_transformer = hist.history

In [ ]:
import pickle

hist_roberta = hist.history
with open("/trainHistoryDict","wb") as file_pi:
    pickle.dump(hist_roberta, file_pi)

In [ ]:
hist_df = pd.DataFrame(hist.history)

In [ ]:
hist_csv_file = 'transformer-history.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

In [ ]:
pd.read_csv("./transformer-history.csv")

In [ ]:
# 여기까지 테스트 코드

In [ ]:
tk.word_index
list(tk.word_index.items())[:10]

In [ ]:
tk.word_index['친쿡부대찌개']

In [ ]:
tk.word_counts
list(tk.word_counts.items())[:10]

In [ ]:
tk.word_index['주']

In [ ]:
seq_data = tk.texts_to_sequences(input_data['shop_name'])

In [ ]:
input_data['shop_name']
# 정수인코딩 작업 진행

In [ ]:
print(seq_data[:100])

In [ ]:
print(tk.word_index['친쿡부대찌개'])
print(tk.word_index['오류동본점'])
print(tk.word_index['gs25'])
print(tk.word_index['씨유'])
print(tk.word_index['세븐일레븐'])

In [ ]:
input_data[:5]

In [ ]:
print(len(seq_data[0]))
print(len(seq_data[1]))
print(len(seq_data[2]))
print(len(seq_data[3]))

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

pad_seq_data = pad_sequences(seq_data)

In [ ]:
print(len(pad_seq_data[0]))
print(len(pad_seq_data[1]))
print(len(pad_seq_data[2]))
print(len(pad_seq_data[3]))

In [ ]:
pad_seq_data.shape

In [ ]:
#워드 임베딩 시작
#자연언어 -> 정수 인코딩 seq data -> 사이즈통일 padding -> (딥러닝)워드 임베딩

In [ ]:
#임베딩 레이어 input data dim

layer1_input_dim = len(tk.word_index) + 1  # vocab size + dummy
layer1_output_dim = 100
layer1_input_length = pad_seq_data[0].shape[0] # 모델에 넣어주는 문장길이

In [ ]:
len(tk.word_index) + 1

In [ ]:
# Drop Nan Values
X = input_data.dropna()

# Get training data
X = input_data.drop('shop_type', axis = 1)

#  Get target label
y = input_data['shop_type']

In [ ]:
messages = X.copy()

In [ ]:
messages.reset_index(inplace = True)

In [ ]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

nltk.download('stopwords')

In [ ]:
# vocabulray size
voc_size = len(tk.word_index) + 1

In [ ]:
# from tensorflow.keras.preprocessing.text import one_hot

# onehot_repr = [one_hot(words, voc_size) for words in corpus]
# normal_repr = [(words, voc_size) for words in corpus]

In [ ]:
# print("Normal Representation:",normal_repr[0][0])
# print("One Hot Representation:",onehot_repr[0])

In [ ]:
# making all sentences of same length
maxlen = 8

# vocabulray size
vocab_size = len(tk.word_index) + 1

In [ ]:
print("One Hot Representation:",onehot_repr[0])


In [ ]:
from keras.utils.vis_utils import plot_model

plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
len(input_data)

In [ ]:
# train : validation : test = 90000 : 21652 : 20000

#train_X = pad_seq_data[:90000]
#valid_X = pad_seq_data[90000:111652]
#test_X = pad_seq_data[111652:]

#train_y = input_data[y_column][:90000]
#valid_y = input_data[y_column][90000:111652]
#test_y = input_data[y_column][111652:]


In [ ]:
from sklearn.model_selection import train_test_split

target = input_data['shop_type']
data = pad_seq_data
train_X, test_X, train_y, test_y = train_test_split(data, target, test_size = 0.4, random_state = 42, stratify = target)
test_X, valid_X, test_y, valid_y = train_test_split(test_X, test_y, random_state = 42, test_size=0.5, stratify = test_y)



In [ ]:
display(train_X, train_y)

In [ ]:
print(train_X.shape)
print(train_y.shape)
print(valid_X.shape)
print(valid_y.shape)
print(test_X.shape)
print(test_y.shape)

In [ ]:
input_data['shop_type'].value_counts()

In [ ]:
test_y.value_counts()

In [ ]:
train_y

In [ ]:
valid_y

In [ ]:
hist.history

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize = (20,10))
plt.plot(hist.history['loss'], label = 'train loss')
plt.plot(hist.history['val_loss'], label = 'validation loss')
plt.xticks(np.arange(0, n_epochs, 1))
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend(loc = 'lower right')
plt.show

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize = (20,10))
ax = plt.axes()
ax.set_xticks(np.arange(0, n_epochs, 1))
ax.set_xticklabels(np.arange(1, n_epochs + 1, 1))
plt.plot(hist.history['accuracy'], label = 'train accuracy')
plt.plot(hist.history['val_accuracy'], label = 'validation accuracy')
plt.xticks(np.arange(0, n_epochs, 1))
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend(loc = 'lower right')
plt.show

In [ ]:
# n_batches = 64
# n_epochs = 2
# model = create_model()
# hist = model.fit(train_X, train_y, validation_data=(valid_X, valid_y),
#                  epochs = n_epochs, batch_size = n_batches)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize = (20,10))
ax = plt.axes()
ax.set_xticks(np.arange(0, n_epochs, 1))
ax.set_xticklabels(np.arange(1, n_epochs + 1, 1))
plt.plot(hist.history['accuracy'], label = 'train accuracy')
plt.plot(hist.history['val_accuracy'], label = 'validation accuracy')
plt.xticks(np.arange(0, n_epochs, 1))
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend(loc = 'lower right')
plt.show

In [ ]:
input_data

In [ ]:
pred_y_prob = model.predict(test_X)
pred_y_prob

In [ ]:
pred_y = np.argmax(pred_y_prob, axis = 1)
pred_y

In [ ]:
true_y = test_y
true_y

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(true_y, pred_y))